<a href="https://colab.research.google.com/github/Nickimani/Airbnb_Project/blob/davis-eda/airbnb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Reverse geoencoder
!pip install reverse_geocoder
import reverse_geocoder as rg


%matplotlib inline

In [4]:
listings_df = pd.read_csv('/content/drive/Shareddrives/CapStone/Data/listings.csv')
reviews_df = pd.read_csv('/content/drive/Shareddrives/CapStone/Data/reviews.csv')
metadata_df = pd.read_csv('/content/drive/Shareddrives/CapStone/Data/data_dictionary.csv')

In [5]:
# listings df

print('Listings dataframe shape: ',listings_df.shape)
print('-'*100)
print(listings_df.info())

Listings dataframe shape:  (18903, 74)
----------------------------------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18903 entries, 0 to 18902
Data columns (total 74 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            18903 non-null  int64  
 1   listing_url                                   18903 non-null  object 
 2   scrape_id                                     18903 non-null  int64  
 3   last_scraped                                  18903 non-null  object 
 4   name                                          18903 non-null  object 
 5   description                                   18560 non-null  object 
 6   neighborhood_overview                         11610 non-null  object 
 7   picture_url                                   18903 non-null  object 
 

In [6]:
null_counts = (listings_df.isna().sum() / len(listings_df)) * 100
null_counts[null_counts > .1]

description                       1.814527
neighborhood_overview            38.581178
host_location                     0.481405
host_about                       45.357880
host_response_time               22.181664
host_response_rate               22.181664
host_acceptance_rate             12.490081
host_neighbourhood               97.275565
neighbourhood                    38.581178
neighbourhood_group_cleansed    100.000000
bathrooms                       100.000000
bathrooms_text                    0.216897
bedrooms                          6.290007
beds                              1.502407
calendar_updated                100.000000
first_review                     27.207322
last_review                      27.207322
review_scores_rating             27.207322
review_scores_accuracy           28.016717
review_scores_cleanliness        28.011427
review_scores_checkin            28.032587
review_scores_communication      28.016717
review_scores_location           28.032587
review_scor

In [7]:
listings_df.head()

,id,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,host_url,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,30332317,https://www.airbnb.com/rooms/30332317,20220622004134,2022-06-22,Riverside cottage,Riverside cottage is a comfortable unit with v...,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,98554302,https://www.airbnb.com/users/show/98554302,...,5.0,5.00,5.0,NaN,t,7,7,0,0,0.07
1,9126484,https://www.airbnb.com/rooms/9126484,20220622004134,2022-06-22,Jubilee Square Apartment,Comfortably furnished 1st floor 2 bedroom apar...,The apartment is situated on the main street d...,https://a0.muscache.com/pictures/59939b2c-967a...,3410651,https://www.airbnb.com/users/show/3410651,...,5.0,4.93,5.0,NaN,f,1,1,0,0,0.19
2,44482125,https://www.airbnb.com/rooms/44482125,20220622004134,2022-06-22,Willow Cottage,Willow cottage overlooks our rose gardens and ...,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,98554302,https://www.airbnb.com/users/show/98554302,...,NaN,NaN,NaN,NaN,t,7,7,0,0,NaN
3,12105523,https://www.airbnb.com/rooms/12105523,20220622004134,2022-06-22,Bush Room,This room is our biggest value for money room....,NaN,https://a0.muscache.com/pictures/939c54ec-4869...,58287766,https://www.airbnb.com/users/show/58287766,...,NaN,NaN,NaN,NaN,t,5,5,0,0,NaN
4,14921766,https://www.airbnb.com/rooms/14921766,20220622004134,2022-06-22,Seabiscuit Farm Cottage with spectacular views,"My place is close to the beach, the airport, f...",NaN,https://a0.muscache.com/pictures/0c24410a-c3d8...,58287766,https://www.airbnb.com/users/show/58287766,...,5.0,5.00,5.0,NaN,t,5,5,0,0,0.02


In [8]:
# reviews df
print('Reviews dataframe shape: ', reviews_df.shape)
print('')
reviews_df.info()

Reviews dataframe shape:  (363065, 6)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 363065 entries, 0 to 363064
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   listing_id     363065 non-null  int64 
 1   id             363065 non-null  int64 
 2   date           363065 non-null  object
 3   reviewer_id    363065 non-null  int64 
 4   reviewer_name  363065 non-null  object
 5   comments       363018 non-null  object
dtypes: int64(3), object(3)
memory usage: 16.6+ MB


In [9]:
reviews_df.isna().sum()

listing_id        0
id                0
date              0
reviewer_id       0
reviewer_name     0
comments         47
dtype: int64

In [10]:
reviews_df.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,30332317,597314314,2020-01-25,7812767,Alan,Lovely rustic accommodation in a picturesque r...
1,30332317,532436007654450454,2022-01-04,367340518,Courtney,The hosts are friendly and takes good care of ...
2,9126484,63063726,2016-02-20,50272994,Carla,6 St Georges Building was lovely in every way....
3,9126484,67195563,2016-03-27,59937103,Gau,"Our host Bruce was great. Responsive, helpful ..."
4,9126484,68077701,2016-04-02,19226497,Imke,Bruce was incredibly welcoming and very flexib...


In [ ]:
# id column is the id of the review

In [ ]:
# price and adjusted price seem to be the same entries

# EDA

## 1. Listings Dataframe

In [11]:
print('Listings dataFrame shape: ', listings_df.shape)
print('')
listings_df.info()

Listings dataFrame shape:  (18903, 74)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18903 entries, 0 to 18902
Data columns (total 74 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            18903 non-null  int64  
 1   listing_url                                   18903 non-null  object 
 2   scrape_id                                     18903 non-null  int64  
 3   last_scraped                                  18903 non-null  object 
 4   name                                          18903 non-null  object 
 5   description                                   18560 non-null  object 
 6   neighborhood_overview                         11610 non-null  object 
 7   picture_url                                   18903 non-null  object 
 8   host_id                                       18903 non-null  int64  
 9   host_url             

In [12]:
listings_df.select_dtypes(include= ["int64", "float64"]).shape[1]

41

In [13]:
listings_df.select_dtypes(include= "object").shape[1]

33

The dataFrame has 18903 rows with 74 columns. 41 of the columns are of numeric type and The remaining 33 are of object type.

In [14]:
null_counts[null_counts > 0].sort_values(ascending= False)

neighbourhood_group_cleansed    100.000000
license                         100.000000
bathrooms                       100.000000
calendar_updated                100.000000
host_neighbourhood               97.275565
host_about                       45.357880
neighborhood_overview            38.581178
neighbourhood                    38.581178
review_scores_checkin            28.032587
review_scores_location           28.032587
review_scores_value              28.032587
review_scores_accuracy           28.016717
review_scores_communication      28.016717
review_scores_cleanliness        28.011427
review_scores_rating             27.207322
last_review                      27.207322
first_review                     27.207322
reviews_per_month                27.207322
host_response_time               22.181664
host_response_rate               22.181664
host_acceptance_rate             12.490081
bedrooms                          6.290007
description                       1.814527
beds       

In [15]:
print('The number of columns with missing values are: ', len(null_counts[null_counts > 0]))

The number of columns with missing values are:  35


The columns neighbourhood_group_cleansed, license, bathrooms, calendar_updated and host_neighbourhood have the highest number of missing values with over 95% of the data missing. We don't think there is much we can do with this columns and we will look to drop them.

### host_neigbourhood

In [16]:
listings_df['host_neighbourhood'].value_counts()

Gwarinpa                        445
Bedford-Stuyvesant               10
Quaid Block                       6
Wandsworth                        6
Al-Wathba North                   4
Downtown Vancouver                3
Shepherd's Bush                   3
Mitte                             2
High Riding Estate                2
Eastside                          2
Chelsea                           2
Old Jaffa                         2
The West End                      2
Knightsbridge                     2
Central LA                        1
Hackney                           1
Westwood Terrace                  1
Forest Glade                      1
Nice                              1
Moabit                            1
Stratland Estates                 1
Portruan                          1
Marylebone                        1
Chinatown                         1
LB of Barnet                      1
Flamingo Vlei                     1
Greenwich                         1
Jumeirah 3                  

In [58]:
dropped = ['neighbourhood_group_cleansed', 'license', 'bathrooms', 'calendar_updated']
listings_df.drop(columns= dropped, inplace= True)
# host_neigbourhood could be filled using or find musipality reverse_geocording

In [ ]:
# filling nan in the host column with Unavailable
listings_df['host_about'] = listings_df['host_about'].fillna('unavailable')

In [ ]:
listings_df['host_about'].unique()

array(['unavailable',
       "Hi there, I was born and still live in Cape Town, arguably the most beautiful city in the world. I grew up in Kalk Bay, a small fishing village on the False Bay coast. Simon's Town is only a 15 minute drive away. I enjoy travelling and exploring new places. Looking forward to meeting you soon!\nCheers Bruce",
       'Hi, I am an American lady who has lived in Zimbabwe and South Africa for the last 22 years. I live in a large home on a flower farm with 2 full time staff,  my 3 English Bull Terrier dogs and 20 chickens.  My 20 year old son is attending college in the USA. I am retired and love the bucolic area where I live. I always have the radio on the a local classical and jazz station and  often entertain friends on my stoop. ',
       ...,
       "I'm a Cape Town doctor lucky to live right on the world's best beach( not just my opinion) in one of the vibiest cities in the world\nWe travel a lot and when we're away our apartment is available(we in turn a

Testing location filler

In [17]:
listings_df[["longitude", "latitude"]].head()

,longitude,latitude
0,18.82662,-34.03002
1,18.43270,-34.19248
2,18.82758,-34.02830
3,18.82689,-34.03149
4,18.82356,-34.03003


In [52]:
v= rg.search((-34.03149, 18.82689))

In [34]:
type(v)

list

In [53]:
v[0]["admin2"]

'Cape Winelands District Municipality'

In [49]:
test = listings_df[["host_neighbourhood", "latitude", "longitude"]]

# def locate(test, "latitude", "longitude"):
#   location = rg.search((test["latitude"][0], test["longitude"][0]))
#   return location[0]["admin2"]
test.head()

,host_neighbourhood,latitude,longitude
0,NaN,-34.03002,18.82662
1,NaN,-34.19248,18.43270
2,NaN,-34.02830,18.82758
3,NaN,-34.03149,18.82689
4,NaN,-34.03003,18.82356


In [57]:
# test["municipal"] = rg.search((test.latitude, test.longitude))
# test.head()

### neighbourhood_overview & neigbourhood

In [66]:
listings_df['neighborhood_overview'].isna().sum()

7293

In [67]:
listings_df['neighborhood_overview'].head()

0                                                  NaN
1    The apartment is situated on the main street d...
2                                                  NaN
3                                                  NaN
4                                                  NaN
Name: neighborhood_overview, dtype: object

In [68]:
listings_df['neighbourhood'].isna().sum()

7293

In [69]:
listings_df['neighbourhood'].head()

0                                      NaN
1    Cape Town, Western Cape, South Africa
2                                      NaN
3                                      NaN
4                                      NaN
Name: neighbourhood, dtype: object

From above, we can see that the column containsinformation about the general location of the airbnb listing. We could use NLP to identify key words which are unique to a particular location. For the neighboorhood column, we could come up with a column which contains the local municipalities in Cape Town and use this column to try and identify significant locations.

### review_scores_checkin, review_scores_location, review_scores_value, review_scores_accuracy, review_scores_communication, review_scores_cleanliness & review_scores_ratings

In [97]:
# Creating a list of the reviews
review_list = list(null_counts[null_counts > 0].sort_values(ascending= False).index[8:18])
review_list

['review_scores_checkin',
 'review_scores_location',
 'review_scores_value',
 'review_scores_accuracy',
 'review_scores_communication',
 'review_scores_cleanliness',
 'review_scores_rating',
 'last_review',
 'first_review',
 'reviews_per_month']

In [85]:
listings_df[review_list].isna().sum()

review_scores_checkin          5299
review_scores_location         5299
review_scores_value            5299
review_scores_accuracy         5296
review_scores_communication    5296
review_scores_cleanliness      5295
review_scores_rating           5143
last_review                    5143
first_review                   5143
reviews_per_month              5143
dtype: int64

In [87]:
listings_df[review_list].head()

,review_scores_checkin,review_scores_location,review_scores_value,review_scores_accuracy,review_scores_communication,review_scores_cleanliness,review_scores_rating,last_review,first_review,reviews_per_month
0,5.0,5.00,5.0,5.0,5.0,5.0,5.00,2022-01-04,2020-01-25,0.07
1,5.0,4.93,5.0,5.0,5.0,5.0,4.87,2022-04-09,2016-02-20,0.19
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5.0,5.00,5.0,5.0,5.0,5.0,5.00,2018-03-25,2018-03-25,0.02


In [88]:
listings_df[review_list].tail()

,review_scores_checkin,review_scores_location,review_scores_value,review_scores_accuracy,review_scores_communication,review_scores_cleanliness,review_scores_rating,last_review,first_review,reviews_per_month
18898,4.89,5.00,4.79,4.95,4.89,4.95,4.79,2022-06-06,2021-08-18,1.84
18899,5.00,5.00,4.00,5.00,4.00,5.00,5.00,2022-04-12,2022-04-12,0.42
18900,4.80,4.40,4.50,4.80,4.70,4.70,4.80,2022-04-18,2021-10-04,1.15
18901,5.00,5.00,5.00,5.00,5.00,5.00,5.00,2022-03-12,2022-03-12,0.29
18902,4.55,4.95,4.59,4.77,4.73,4.77,4.59,2021-04-30,2017-12-05,0.40


All columns are of numeric type exept for the last_review and first_review columns which are of date type. We feel this columns wont be of much help in our objectives therefore we will look to drop these columns.

In [94]:
listings_df[review_list].describe().T

,count,mean,std,min,25%,50%,75%,max
review_scores_checkin,13604.0,4.808905,0.462444,0.00,4.81,4.97,5.00,5.00
review_scores_location,13604.0,4.803673,0.414366,0.00,4.78,4.93,5.00,5.00
review_scores_value,13604.0,4.692829,0.504996,0.00,4.63,4.83,5.00,5.00
review_scores_accuracy,13607.0,4.757262,0.490736,0.00,4.73,4.91,5.00,5.00
review_scores_communication,13607.0,4.802807,0.479074,0.00,4.81,4.97,5.00,5.00
review_scores_cleanliness,13608.0,4.730264,0.492878,0.00,4.67,4.89,5.00,5.00
review_scores_rating,13760.0,4.666231,0.684017,0.00,4.64,4.86,5.00,5.00
reviews_per_month,13760.0,0.830200,1.084783,0.01,0.15,0.45,1.09,18.86


We have significant percentage of missing values in some of the columns, We will impute the median of these columns into the missing values. We do that below:

In [95]:
# imputing missing values
listings_df[review_list] = listings_df[review_list].fillna(listings_df[review_list].median())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  


In [96]:
listings_df[review_list].isna().sum()

review_scores_checkin             0
review_scores_location            0
review_scores_value               0
review_scores_accuracy            0
review_scores_communication       0
review_scores_cleanliness         0
review_scores_rating              0
last_review                    5143
first_review                   5143
reviews_per_month                 0
dtype: int64

In [100]:
# Dropping the columns below
listings_df.drop(columns= review_list[-3:-1], inplace= True)

#### Questions?
1. What is the distribution of review scores?
2. what is the relationship between review scores and superhosts?

In [84]:
null_counts[null_counts > 0].sort_values(ascending= False)

neighbourhood_group_cleansed    100.000000
license                         100.000000
bathrooms                       100.000000
calendar_updated                100.000000
host_neighbourhood               97.275565
host_about                       45.357880
neighborhood_overview            38.581178
neighbourhood                    38.581178
review_scores_checkin            28.032587
review_scores_location           28.032587
review_scores_value              28.032587
review_scores_accuracy           28.016717
review_scores_communication      28.016717
review_scores_cleanliness        28.011427
review_scores_rating             27.207322
last_review                      27.207322
first_review                     27.207322
reviews_per_month                27.207322
host_response_time               22.181664
host_response_rate               22.181664
host_acceptance_rate             12.490081
bedrooms                          6.290007
description                       1.814527
beds       

In [64]:
listings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18903 entries, 0 to 18902
Data columns (total 70 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            18903 non-null  int64  
 1   listing_url                                   18903 non-null  object 
 2   scrape_id                                     18903 non-null  int64  
 3   last_scraped                                  18903 non-null  object 
 4   name                                          18903 non-null  object 
 5   description                                   18560 non-null  object 
 6   neighborhood_overview                         11610 non-null  object 
 7   picture_url                                   18903 non-null  object 
 8   host_id                                       18903 non-null  int64  
 9   host_url                                      18903 non-null 